In [2]:
import pandas as pd
import plotly.express as px

In [54]:
athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['result_id','athlete','athlete_id','pos'])


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1

athlete_results.drop(repeat, inplace=True)
athlete_results.drop(columns=['edition_id', 'isTeamSport', 'index'], inplace=True)
athlete_results['edition'] = athlete_results['edition'].str.split().str[0].astype(int)

athlete_results["medal"] = athlete_results["medal"].notnull().mul(1)

gr = pd.DataFrame(athlete_results.groupby(['country_noc', 'sport', 'edition', 'event']).sum()).reset_index()


In [74]:
country = 'USA'
dataset = gr[gr['country_noc'] == country].reset_index()


new_medal = []
for i in range(len(dataset)):
    edition = dataset.loc[i, 'edition']
    sport = dataset.loc[i, 'sport']
    event = dataset.loc[i, 'event']

    new_medal.append(dataset[(dataset['edition'] <= edition) & (dataset['sport'] == sport) & (dataset['event'] == event)]['medal'].sum())

sports = pd.unique(dataset.sport)

dataset['medal'] = new_medal


In [75]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('Sports evolution of ' + country),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=sports,
        value=[sports[0]],
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))

def update_line_chart(s):
    df = dataset#px.data.gapminder()
    mask = df.sport.isin(s)
    fig = px.line(df[mask], 
        x="edition", y="medal", color='event')
    return fig


app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
